# imports

In [41]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split 
import pandas as pd
import numpy as np

# Loading file
Loding file finally created by _'Data Visualization and Preprocessing'_

In [42]:
data  =  pd.read_csv('final_data.csv')

In [43]:
data.head()

,Day_name,Time,Name
0,Wednesday,1105,ct
1,Wednesday,1028,ct
2,Wednesday,1028,sd
3,Tuesday,1032,sd
4,Monday,2328,sk


# Pre-processing
STEP-I: Encode data which are str type amd OneHot encode name 

STEP-II: Cyclic Normalize Day and hour coloumn

In [44]:
# STEP I :
Day_name_dict = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 
                 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}
data['Day_name'] =  [i for i in map(lambda x: Day_name_dict[x], data['Day_name'])]

In [45]:
enc = preprocessing.OneHotEncoder()
enc.fit(data['Name'].values.reshape(-1,1))
y = enc.transform(data['Name'].values.reshape(-1,1))

In [46]:
data.head()

,Day_name,Time,Name
0,2,1105,ct
1,2,1028,ct
2,2,1028,sd
3,1,1032,sd
4,0,2328,sk


In [47]:
#STEP II
data['Day_sin'] = np.sin(data['Day_name'])
data['Day_cos'] = np.cos(data['Day_name'])
data['Time_sin'] = np.sin(data['Time'])
data['Time_cos'] = np.cos(data['Time'])

In [48]:
data.head()

,Day_name,Time,Name,Day_sin,Day_cos,Time_sin,Time_cos
0,2,1105,ct,0.909297,-0.416147,-0.745053,0.667005
1,2,1028,ct,0.909297,-0.416147,-0.643607,-0.765356
2,2,1028,sd,0.909297,-0.416147,-0.643607,-0.765356
3,1,1032,sd,0.841471,0.540302,0.999913,0.013186
4,0,2328,sk,0.000000,1.000000,-0.079759,-0.996814


We want to maintain the cyclic nature of _Day_ and _Time_ feature. Therefore spliting each column into sin and cos and plotting it wrt circle.

In [49]:
X = data.as_matrix(columns=['Day_sin', 'Day_cos', 'Time_sin', 'Time_cos'])

D:\Anaconda3\envs\env-tf-gpu\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


# Spliting training and test data
20% of test data is seperated from orginal data. And the data is shuffled by default.

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2)

In [65]:
X.shape, y.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape

((235, 4), (235, 19), (188, 4), (47, 4), (188, 19), (47, 19))

# Model Training

In [66]:
model = KNeighborsClassifier()
model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

# Model Prediction

For now we are just considering 3 nearest neighbour of a test point. And if any one of the neighbour matches with the actual data, we will consider it right answer. 

In [67]:
# Getting 3 nearest neighbour
res = model.kneighbors(X=X_test, n_neighbors=3, return_distance=False)

In [68]:
"""
The res contain the indexes(wrt y_train) of nearest neighbour.
Therefore, first we have to access the value. These values are one-Hot encoded,
so we have to inverse transform it.

"""
result = [enc.inverse_transform([y_train[i] for i in res][j]).reshape(-1,3) for j in range(len(res))]

Calculation for Accuracy.
For now taking only considering accuracy.

In [69]:
total = 0
right = 0 
for i in range(len(res)):
    total+=1 
    if enc.inverse_transform(y_test)[i] in result[i]:
        right+=1

In [70]:
print("Model predicted {} correct out {}".format(right, total))

Model predicted 25 correct out 47


In [71]:
print("Accuracy: {0:.2f}".format(right/total*100))

Accuracy: 53.19


# End

__What if:-__

1) Consider more/less then 3 nearest 